# Chapter 10 - Implementing Deep Learning (DL) and Convolutional Neural Network (CNN)

In [2]:
import os
import random
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

#import warnings
#warnings.filterwarnings("ignore")

### Create the Dataset

Source: https://www.microsoft.com/en-us/download/details.aspx?id=54765

In [4]:
# Set paths for the dataset
cat_path = './dogs-vs-cats/cats'
dog_path = './dogs-vs-cats/dogs'

# Prepare lists for cat and dog image file paths
cat_images = [os.path.join(cat_path, img) for img in os.listdir(cat_path) if img.endswith('.jpg')]
dog_images = [os.path.join(dog_path, img) for img in os.listdir(dog_path) if img.endswith('.jpg')]

# Combine all images and create labels (0 for cat, 1 for dog)
all_images = cat_images + dog_images
all_labels = [0] * len(cat_images) + [1] * len(dog_images)

# Shuffle the dataset and labels together
combined = list(zip(all_images, all_labels))
random.shuffle(combined)
all_images, all_labels = zip(*combined)

# Split the dataset into training and testing sets (80% train, 20% test)
train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

# Create directories to store the training and testing data
base_train_path = './dogs-vs-cats/train'
base_test_path = './dogs-vs-cats/test'

for folder in [base_train_path, base_test_path]:
    for label in ['cat', 'dog']:
        os.makedirs(os.path.join(folder, label), exist_ok=True)

# Move images to the train and test directories based on labels
for img, label in zip(train_images, train_labels):
    label_folder = 'cat' if label == 0 else 'dog'
    shutil.copy(img, os.path.join(base_train_path, label_folder))

for img, label in zip(test_images, test_labels):
    label_folder = 'cat' if label == 0 else 'dog'
    shutil.copy(img, os.path.join(base_test_path, label_folder))

print("Train and Test directories are ready with images organized by category.")

Train and Test directories are ready with images organized by category.


### Cleaning the Dataset

In [6]:
import os
import warnings
from PIL import Image, UnidentifiedImageError

# Define the directories to check
directories = ['./dogs-vs-cats/train/cat', './dogs-vs-cats/train/dog', './dogs-vs-cats/test/cat', './dogs-vs-cats/test/dog']

# Iterate through each directory to check and clean images
for i in range(3):  # Run multiple passes to ensure all corrupted images are removed
    for directory in directories:
        for filename in os.listdir(directory):
            filepath = os.path.join(directory, filename)
            try:
                # Ignore warnings during image processing
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    with Image.open(filepath) as img:  # Try opening the image file
                        img.verify()                   # Verify if it's an image
                        img = Image.open(filepath)     # Open it again to try loading the image properly
                        img.load()                     # Ensure that the entire image is loaded properly
            except (IOError, SyntaxError, UnidentifiedImageError) as e:
                # Remove the problematic file if an error occurs during verification or loading
                print(f"Removing problematic image: {filepath}")
                os.remove(filepath)

print("Image cleaning process completed.")

Removing problematic image: ./dogs-vs-cats/train/cat\10744.jpg_temp.jpg
Image cleaning process completed.


### Preparing the Training and Test Datasets

In [9]:
from keras.preprocessing.image import ImageDataGenerator

# Define constants for image size
Image_Width = 128                          # Set the width of the images to 128 pixels
Image_Height = 128                         # Set the height of the images to 128 pixels
Image_Size = (Image_Width, Image_Height)   # Define the image size as a tuple (128, 128)
batch_size = 32                            # Set the batch size to 32 images per batch

# Initialize ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)  # Rescale images to normalize pixel values between 0 and 1
test_datagen = ImageDataGenerator(rescale=1./255)   # Rescale testing images to normalize pixel values between 0 and 1

# Create training and testing sets using flow_from_directory
training_set = train_datagen.flow_from_directory(
    './dogs-vs-cats/train',                # Path to the training dataset directory
    target_size=Image_Size,                # Resize all images to (128, 128)
    batch_size=batch_size,                 # Load images in batches of 32
    class_mode='binary'                    # Define class mode as 'binary' since there are only two classes (cat and dog)
)

test_set = test_datagen.flow_from_directory(
    './dogs-vs-cats/test',                 # Path to the testing dataset directory
    target_size=Image_Size,                # Resize all images to (128, 128)
    batch_size=batch_size,                 # Load images in batches of 32
    class_mode='binary'                    # Define class mode as 'binary' since there are only two classes (cat and dog)
)

print("Training and Test sets are ready for model training.")  # Print a message indicating that the training and testing sets are ready

Found 25003 images belonging to 2 classes.
Found 23631 images belonging to 2 classes.
Training and Test sets are ready for model training.


### Create the Neural Net Model

In [11]:
from keras.models import Sequential                            # Import Sequential to initialize a sequential neural network model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense  # Import layers for building the CNN model

# Initializing the CNN model
model = Sequential()                        # Create a sequential model which allows adding layers one by one

# Adding Convolution Layer
model.add(Conv2D(32,                        # Add a 2D convolutional layer with 32 filters
                 (3, 3),                    # Each filter is 3x3 in size
                 input_shape=(128, 128, 3), # Input shape is set to (128, 128, 3) for 128x128 RGB images
                 activation='relu'))        # Use ReLU activation to introduce non-linearity

# Adding Pooling Layer
model.add(MaxPooling2D(pool_size=(2, 2)))   # Add a max pooling layer with pool size 2x2 to reduce dimensionality

# Adding another Convolution Layer
model.add(Conv2D(32,                        # Add another 2D convolutional layer with 32 filters
                 (3, 3),                    # Each filter is 3x3 in size
                 activation='relu'))        # Use ReLU activation to introduce non-linearity again

model.add(MaxPooling2D(pool_size=(2, 2)))   # Add another max pooling layer with pool size 2x2

# Flattening
model.add(Flatten())                        # Flatten the pooled feature maps to create a single feature vector

# Full Connection (Fully Connected Layer)
model.add(Dense(units=128,                  # Add a fully connected layer with 128 neurons
                activation='relu'))         # Use ReLU activation

model.add(Dense(units=1,                    # Add an output layer with 1 neuron (for binary classification)
                activation='sigmoid'))      # Use sigmoid activation to get a probability output between 0 and 1

# Compiling the CNN
model.compile(optimizer='adam',             # Compile the model using the Adam optimizer
              loss='binary_crossentropy',   # Use binary crossentropy as the loss function (since it’s a binary classification problem)
              metrics=['accuracy'])         # Track the accuracy of the model during training

# Fitting the CNN to the Training Set
model.fit(training_set,                     # Train the model using the training data
          steps_per_epoch=625,              # Number of batches per epoch (20000 images / batch_size 32 = 625)
          epochs=10,                        # Train the model for 10 epochs
          validation_data=test_set,         # Use the test set for validation during training
          validation_steps=157)             # Number of validation batches per epoch (5000 images / batch_size 32 ≈ 157)

Epoch 1/10
625/625 [==============================] - 170s 270ms/step - loss: 0.6087 - accuracy: 0.6639 - val_loss: 0.5087 - val_accuracy: 0.7464
Epoch 2/10
625/625 [==============================] - 171s 273ms/step - loss: 0.4770 - accuracy: 0.7686 - val_loss: 0.4614 - val_accuracy: 0.7703
Epoch 3/10
625/625 [==============================] - 169s 271ms/step - loss: 0.4097 - accuracy: 0.8144 - val_loss: 0.3461 - val_accuracy: 0.8529
Epoch 4/10
625/625 [==============================] - 174s 279ms/step - loss: 0.3330 - accuracy: 0.8539 - val_loss: 0.2654 - val_accuracy: 0.8925
Epoch 5/10
625/625 [==============================] - 177s 283ms/step - loss: 0.2542 - accuracy: 0.8963 - val_loss: 0.1628 - val_accuracy: 0.9445
Epoch 6/10
625/625 [==============================] - 170s 273ms/step - loss: 0.1801 - accuracy: 0.9305 - val_loss: 0.1469 - val_accuracy: 0.9494
Epoch 7/10
625/625 [==============================] - 170s 272ms/step - loss: 0.1236 - accuracy: 0.9534 - val_loss: 0.0826 -

### Summary

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 28800)             0         
                                                                 
 dense (Dense)               (None, 128)               3

#### Notes:

**Accuracy:** Your model's accuracy started at around 66% and ended up at around 99.36% after 10 epochs, which is excellent. The model is now able to predict correctly for almost all the training images.

**Validation Accuracy:** The validation accuracy has also significantly improved, reaching around 99.24%, indicating that the model generalizes well to the unseen test data.

**Loss:** Both training and validation loss decreased consistently, indicating that the model is effectively learning from the data.

### Saving the model

In [16]:
# Save the trained model
#model.save('cats_vs_dogs_model.h5')  

### Make predictions:

In [18]:
from keras.utils import load_img, img_to_array

def predict_image(model, image_path):
    
    # Load an image to make a prediction
    img = load_img(image_path, target_size=(128, 128))  # Load and resize the image
    img = img_to_array(img)                             # Convert the image to a NumPy array
    img = img / 255.0                                   # Normalize pixel values to be between 0 and 1
    img = img.reshape(1, 128, 128, 3)                   # Reshape to add batch dimension

    # Make prediction
    prediction = model.predict(img)
    if prediction[0][0] > 0.5:
        return "It's a Dog!"
    else:
        return "It's a Cat!"

```Python
### Loading Model for prediction
from keras.models import load_model

# Load the trained model
model = load_model('cats_vs_dogs_model.h5')

# Predict an image using the loaded model
result = predict_image(model, 'path/to/your/image.jpg')
print(result)
```

In [20]:
# Example usage
result = predict_image(model, './dogs-vs-cats/dogs/39.jpg')
print(result)

1/1 [==============================] - 0s 167ms/step
It's a Dog!
